# A multivariate regression problem

In [1]:
import pandas
import numpy as np
import sklearn.linear_model as lm
from sklearn.model_selection import KFold
from sklearn import preprocessing as pre
from sklearn.decomposition import PCA
import random
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

Load in the California housing dataset. Originally downloaded from https://github.com/ageron/handson-ml/tree/master/datasets/housing

In [2]:
housing = pandas.read_csv('data/housing.csv')

This dataset has some special features which need some attention:
1. The variable ocean_proximity is a categorical variable
2. There are some missing values in the total_bedrooms column
Let us consider how to deal with each of these.

First, deal with the categorical variable ocean_proximity. How many distinct values does this take?

In [3]:
print(set(housing.ocean_proximity.values))
print(len(set(housing.ocean_proximity.values)))

{'ISLAND', '<1H OCEAN', 'INLAND', 'NEAR BAY', 'NEAR OCEAN'}
5


There are five unique values and so we will replace this with a one-hot vector of length 5 where 10000 corresponds to <1H OCEAN, 01000 is ISLAND etc. This is a standard way to represent categorical factors. The key is that for any entry (row) only one element of this vector should be 1 - hence one-hot.

In [4]:
housing['1h_ocean'] = [1 if i=='<1H OCEAN' else 0 for i in housing.ocean_proximity.values]
housing['island'] = [1 if i=='ISLAND' else 0 for i in housing.ocean_proximity.values]
housing['inland'] = [1 if i=='INLAND' else 0 for i in housing.ocean_proximity.values]
housing['near_ocean'] = [1 if i=='NEAR OCEAN' else 0 for i in housing.ocean_proximity.values]
housing['near_bay'] = [1 if i=='NEAR BAY' else 0 for i in housing.ocean_proximity.values]
housing.drop(columns=['ocean_proximity'], inplace=True)

Now we need to look at the missing values in total_bedrooms, of which there are:

In [5]:
sum(housing.total_bedrooms.isna())

207

How can we deal with this? It depends. There are several strategies one can use.

* Replace with the average of the column, but this loses information about correlation
* Replace with the values from the nearest neighbour, based on the values of the other variables.
* Use some prior knowledge.

We will use some prior knowledge that it is highly likely that the number of bedrooms is strongly correlated with the total number of rooms. We'll fit a linear model to predict the missing values.

In [6]:
# Get the non-Nan indices
notna = housing.total_bedrooms.notna()

In [7]:
model = lm.LinearRegression()
model.fit(housing.total_rooms.values[notna].reshape(-1,1), housing.total_bedrooms.values[notna].reshape(-1,1))
model.score(housing.total_rooms.values[notna].reshape(-1,1), housing.total_bedrooms.values[notna].reshape(-1,1))

0.8656060227407114

This is a strong prediction so our intuition is correct. Now we predict the missing values.

In [8]:
isna = housing.total_bedrooms.isna()
missing_bedrooms = model.predict(housing.total_rooms.values[isna].reshape(-1,1))

Insert the imputed values into the table.

In [9]:
# Can ignore subsequent warning
housing.total_bedrooms.loc[isna] = np.squeeze(missing_bedrooms)

/home/david/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


So we now have a complete dataset. Can we model it? An obvious thing to try first is a linear model, in which the house price is a weighted sum of the inputs; that is:
$$y = w_0 + w_1 x_1 + w_2 x_2 + w_M x_M$$
Let's assess this via cross-validation

In [10]:
x = housing.values #returns a numpy array
std_scaler = pre.StandardScaler()
x_scaled = std_scaler.fit_transform(x)
df = pandas.DataFrame(x_scaled)
df.columns = housing.columns

In [11]:
#house_price = df["median_house_value"]
#df.drop("median_house_value", axis=1, inplace=True)

In [12]:
#pca = PCA(n_components=9)
# X is the matrix transposed (n samples on the rows, m features on the columns)
#pca.fit(df)
#new_df = pca.transform(df)
#new_df = pandas.DataFrame(new_df)

In [13]:
#house_price_df = pandas.DataFrame(house_price)
#house_price_df.reset_index(drop=True, inplace=True)
#new_df.reset_index(drop=True, inplace=True)
#new_df["median_house_value"] = house_price_df
#display(new_df)

In [14]:
# First, extract the data into arrays
y = df.median_house_value.values.reshape(-1,1)
X = df.drop(columns=['median_house_value'], inplace=False).values
print(X.shape)
print(y.shape)
# Pull out 1000 values into a holdout set
holdout = random.sample(range(0,10640),1000)
X_holdout = X[holdout]
y_holdout = y[holdout]
Xt = np.delete(X, holdout, 0)
yt = np.delete(y, holdout, 0)
print(Xt.shape)
print(yt.shape)

(20640, 13)
(20640, 1)
(19640, 13)
(19640, 1)


In [15]:
Model = lm.LinearRegression()
# Have to shuffle the data because it is grouped.
train_avg = 0
test_avg = 0
number_of_runs = 0 
kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(Xt):
    number_of_runs = number_of_runs +1
    
    X_train, X_test = Xt[train_index], Xt[test_index]
    y_train, y_test = yt[train_index], yt[test_index]
    Model.fit(X_train, y_train)
    train = Model.score(X_train, y_train)
    test = Model.score(X_test, y_test)
    train_avg += train
    test_avg += test
    
train_avg = (train_avg/number_of_runs)
test_avg = (test_avg/number_of_runs)

print("Training average: " + str(train_avg))
print("Test average: " + str(test_avg))

Training average: 0.647187893112871
Test average: 0.6449187119334369


Scores quoted are $R^2$ (coefficient of determination) values which range from 0 to 1. These are OK but there is much room for improvement and we ought to be able to do much better than this. Options that we could try are:

* Normalising/rescaling the data so that all variables have similar values?
* Expanding the basis to include terms that are non-linear in the variables?
* Removing redundant variables from the data - are there any that are correlated with each other?
* Regularisation?

This will be the task of the group assignment.

In [16]:
steps = [
    ('poly', pre.PolynomialFeatures(degree = 3)),
    ('model', lm.RidgeCV(alphas=(0.1, 1,10), fit_intercept=True))
]
kf = KFold(n_splits=5, shuffle=True)

train_avg = 0
test_avg = 0
number_of_runs = 0 
for train_index, test_index in kf.split(Xt):
  number_of_runs = number_of_runs +1
  X_train, X_test = Xt[train_index], Xt[test_index]
  y_train, y_test = yt[train_index], yt[test_index]
  ridge_pipe = Pipeline(steps)
  ridge_pipe.fit(X_train, y_train)
  train = ridge_pipe.score(X_train, y_train)
  test = ridge_pipe.score(X_test, y_test)
  train_avg += train
  test_avg += test

train_avg = (train_avg/number_of_runs)
test_avg = (test_avg/number_of_runs)

print("Training average: " + str(train_avg))
print("Test average: " + str(test_avg))


Training average: 0.7741684453932557
Test average: 0.45016798991349666
